In [1]:
%pip install tensorflow keras numpy pandas matplotlib scikit-learn keras-tuner building_footprint_segmentation

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 7.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report
from PIL import Image

# Enable mixed precision training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

# Verify GPU availability
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPUs Available:", gpus)
else:
    print("No GPU found. Please ensure that TensorFlow is configured properly.")

# Set the image size and batch size
img_size = (224, 224)
batch_size = 32

# Set the base directory for the data
base_dir = '/content/drive/MyDrive/Madhu RA Work Folder/Img' # Adjust this path as needed

# Function to get class names
def get_class_names(data_directory):
    class_names = sorted([item for item in os.listdir(data_directory) if os.path.isdir(os.path.join(data_directory, item))])
    print("Classes:", class_names)
    return class_names

# Function to collect data paths excluding specified classes
def collect_data_paths(data_directory, excluded_classes, class_names):
    image_paths = []
    labels = []
    included_class_names = [cls for cls in class_names if cls not in excluded_classes]
    class_indices = {class_name: idx for idx, class_name in enumerate(included_class_names)}
    for class_name in included_class_names:
        folder_path = os.path.join(data_directory, class_name)
        if os.path.isdir(folder_path):
            for img_file in os.listdir(folder_path):
                img_path = os.path.join(folder_path, img_file)
                # Filter out non-image files
                if img_file.lower().endswith(('.tif', '.tiff', '.jpg', '.jpeg', '.png', '.bmp', '.gif')):
                    image_paths.append(img_path)
                    labels.append(class_indices[class_name])
    return image_paths, labels, included_class_names

# Function to load and preprocess image with PIL
def load_image_with_pil(filename):
    filename = filename.decode('utf-8')  # Decode the bytes object
    image = Image.open(filename).convert('RGB')
    image = image.resize(img_size)
    image = np.array(image) / 255.0  # Normalize to [0,1]
    return image.astype(np.float32)

# Function to parse and preprocess images using PIL
def parse_function(filename, label):
    # Read image using PIL via tf.numpy_function
    image = tf.numpy_function(load_image_with_pil, [filename], tf.float32)
    image.set_shape([img_size[0], img_size[1], 3])  # Set static shape
    return image, label

# Data augmentation using Keras preprocessing layers
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

# Function to create datasets
def create_dataset(image_paths, labels, training=True):
    labels = np.array(labels)
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(parse_function, num_parallel_calls=tf.data.AUTOTUNE)
    if training:
        dataset = dataset.shuffle(buffer_size=1024)
        dataset = dataset.batch(batch_size)
        dataset = dataset.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)
    else:
        dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

# Function to build the model using best hyperparameters
def build_model(num_classes):
    with tf.device('/GPU:0'):
        base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))
        # Fine-tune starting at the specified layer
        for layer in base_model.layers[:500]:
            layer.trainable = False
        for layer in base_model.layers[500:]:
            layer.trainable = True

        # Input layer
        inputs = Input(shape=(img_size[0], img_size[1], 3))
        x = inputs
        x = base_model(x, training=True)
        x = GlobalAveragePooling2D()(x)
        x = Dense(768, activation='relu')(x)
        x = Dropout(0.4)(x)
        outputs = Dense(num_classes, activation='softmax', dtype='float32')(x)  # Use float32 for final layer

        model = Model(inputs, outputs)

        # Compile the model
        optimizer = Adam(learning_rate=5.838015556029658e-05)
        optimizer = mixed_precision.LossScaleOptimizer(optimizer)
        model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

# Function to train and evaluate the model with excluded classes
def train_and_evaluate(excluded_classes, base_dir, img_size=(224, 224), batch_size=32, epochs=20):
    # Get class names from training directory
    data_dir = os.path.join(base_dir, 'train')
    class_names = get_class_names(data_dir)

    # Collect data paths excluding specified classes
    train_paths, train_labels, included_class_names = collect_data_paths(os.path.join(base_dir, 'train'), excluded_classes, class_names)
    val_paths, val_labels, _ = collect_data_paths(os.path.join(base_dir, 'val'), excluded_classes, class_names)
    test_paths, test_labels, _ = collect_data_paths(os.path.join(base_dir, 'test'), excluded_classes, class_names)

    num_classes = len(included_class_names)
    print(f"Training with classes: {included_class_names}")

    # Create datasets
    train_dataset = create_dataset(train_paths, train_labels, training=True)
    val_dataset = create_dataset(val_paths, val_labels, training=False)
    test_dataset = create_dataset(test_paths, test_labels, training=False)

    # Build the model
    model = build_model(num_classes)

    # Implement early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train the model
    history = model.fit(
        train_dataset,
        epochs=epochs,
        validation_data=val_dataset,
        callbacks=[early_stopping]
    )

    # Save the model
    excluded_classes_str = '_'.join(excluded_classes) if excluded_classes else 'all_classes'
    model.save(f'Densenet201_excluded_{excluded_classes_str}.h5')

    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(test_dataset)
    print(f"Test Accuracy excluding {excluded_classes}: {test_accuracy:.4%}")

    # Generate confusion matrix and classification report
    y_true = np.concatenate([y.numpy() for x, y in test_dataset], axis=0)
    y_pred_probs = model.predict(test_dataset)
    y_pred = np.argmax(y_pred_probs, axis=1)

    # Adjust class names for the included classes
    included_class_indices = {idx: class_name for idx, class_name in enumerate(included_class_names)}

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:\n", cm)

    # Define labels for classification report
    labels = range(num_classes)

    # Classification Report
    cr = classification_report(y_true, y_pred, labels=labels, target_names=included_class_names, zero_division=0)
    print("Classification Report:\n", cr)

    return test_accuracy

# Testing different class exclusions
results = {}

# All classes included
print("\nTraining with all classes included:")
results['All classes'] = train_and_evaluate([], base_dir)

# Excluding each class one by one
classes_to_exclude = ['Commercial', 'High', 'Hospital', 'Industrial', 'Multi', 'Schools', 'Single']

for cls in classes_to_exclude:
    print(f"\nExcluding class: {cls}")
    results[f'No {cls}'] = train_and_evaluate([cls], base_dir)

# Print results
print("\nSummary of Results:")
for key, value in results.items():
    print(f'{key}: {value:.4%}')


GPUs Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

Training with all classes included:


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Madhu RA Work Folder/Img/train'